# Converting the raw ocean-inlandwater-land mask into the Sen2Water mask

This notebook converts the raw mask produced from WorldCover, Global Islands, and GRHHS Continental Coastlines into the final Sen2Water mask. It decides between the three mask using spatial criteria to resolve inconsistencies with the goal of a consistent discrimination of ocean and inland water. The technique is selected buffering.

The final mask encodes distance to the ocean border in the transition area in estuaries. This allows to switch smoothly between two algorithms based on spatial criteria if this is desired. If not, then the distance does not need to be used.

The output is a mask with four main values OCEAN (64), INLANDWATER (92), COASTAL (160), and LAND (192). Pixel distance to ocean in inland water is encoded as values between 65 and 91. Distance to ocean in coastal land at estuaries is encoded as values between 161 and 191. This can be used for switching for pixels dynamically detected as water.

In [ ]:
import rioxarray as rio
import xarray as xr
import numpy as np
import scipy
import math
print("software imported")

In [ ]:
combined_mask_path = "combined_mask-32UME.tif"
s2w_mask_path = "s2w-globalmask-32UME.tif"
countslist_path = "globalmaskcounts.list"

In [ ]:
# define legend of the target mask (and intermediate layers)
INVALID = 0
OCEAN = 64
INLANDWATER = 96
BOTTOMREFLECTION = 128
COASTAL = 160
LAND = 192

RIVERWATER=3
SHORE_TRANSITION = 2
SHORE_INLANDWATER = 1
print("legend defined")

In [ ]:
hr = rio.open_rasterio(combined_mask_path)
hr_width = len(hr.x)
hr_height = len(hr.y)
hr_size = hr_height * hr_width
hr_step = 60.0
hr_left = hr.x.data[0]
hr_top = hr.y.data[0]
raw_mask = hr.data[0]

In [ ]:
mask = np.zeros(raw_mask.shape, dtype=np.uint8)
mask[(raw_mask & 1) != 0] = LAND
mask[(mask == 0) & ((raw_mask & 2) != 0)] = INLANDWATER
mask[(mask == 0) & ((raw_mask & 4) == 0)] = OCEAN
mask[(mask==0)] = RIVERWATER

da = xr.DataArray(mask.reshape((1, *mask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da.plot()

* RIVERWATER = 1  (temporary value)
* OCEAN = 64
* INLANDWATER = 96
* LAND = 192

In [ ]:
FOUR_NEIGHBOURS=np.array([[0,1,0],[1,1,1],[0,1,0]], dtype=np.int8)
EIGHT_NEIGHBOURS=np.array([[1,1,1],[1,1,1],[1,1,1]], dtype=np.int8)

# replace neighbours of value source that are value dest by value insert
def dilate(mask, source_value, neighbours, steps, dest_value, insert_value):
    source_mask = (mask == source_value) | (mask == insert_value)
    dest_mask = mask == dest_value
    dilated = scipy.ndimage.binary_dilation(source_mask, neighbours, steps, dest_mask)
    mask[dest_mask & dilated] = insert_value
    del source_mask, dest_mask, dilated

# replace neighbours of value source that are value dest by value insert
def dilate_range(mask, source_value1, source_value9, neighbours, steps, dest_value, insert_value):
    source_mask = (mask>=source_value1) & (mask<=source_value9)
    dest_mask = mask == dest_value
    dilated = scipy.ndimage.binary_dilation(source_mask, neighbours, steps, dest_mask)
    mask[dest_mask & dilated] = insert_value
    del source_mask, dest_mask, dilated

In [ ]:
print("buffering ocean into 'rivers' and back to preserve ocean accidentally included in continental shoreline ...")

depth = 24  # 32
for i in range(depth):
    dilate(mask, OCEAN, FOUR_NEIGHBOURS, 1, RIVERWATER, OCEAN)

# ... and by reverting in river mouths
for i in range(depth):
    dilate(mask, RIVERWATER, FOUR_NEIGHBOURS, 1, OCEAN, RIVERWATER)

mask[mask==RIVERWATER] = INLANDWATER

print("almost no effect in this granule")

In [ ]:
da = xr.DataArray(mask.reshape((1, *mask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da[:,405:710,1525:].plot()
print("before ...")

In [ ]:
print("buffering ocean into inlandwater and back to extend ocean up to worldcover coastline ...")

# buffer ocean into inlandwater by n pixels
# buffer inlandwater back into ocean in estuaries
depth = 48  # 24
for i in range(depth):
    dilate(mask, OCEAN, FOUR_NEIGHBOURS, 1, INLANDWATER, OCEAN)

# ... and by reverting in river mouths
for i in range(depth):
    dilate(mask, INLANDWATER, FOUR_NEIGHBOURS, 1, OCEAN, INLANDWATER)

da = xr.DataArray(mask.reshape((1, *mask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da[:,405:710,1525:].plot()

In [ ]:
print("buffering ocean into land to mark coastal land ...")

depth = 28
for i in range(depth):
    dilate(mask, OCEAN, EIGHT_NEIGHBOURS, 1, LAND, COASTAL)

da = xr.DataArray(mask.reshape((1, *mask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da[:,405:710,1525:].plot()

In [ ]:
da = xr.DataArray(mask.reshape((1, *mask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da[:,1455:1555,1055:1155].plot()
print("before ...")

In [ ]:
print("mask transition zone from ocean to inland water ...")

# values 64 < v < 96 are the transition zone from ocean to inland water
# incremental values 7+distance from ocean up to 32 pixels
# 4 neighbours for the first two steps to avoid jumps over locks
depth = 32
for i in range(depth):
    dilate(mask, OCEAN+i, FOUR_NEIGHBOURS if i<2 else EIGHT_NEIGHBOURS, 1, INLANDWATER, OCEAN+i+1)

shore_width = 4
transition_depth = 32
for i in range(shore_width):
    if i == 0:
        dilate_range(mask, OCEAN+1, INLANDWATER-1, EIGHT_NEIGHBOURS, 1, COASTAL, SHORE_TRANSITION)
    else:
        dilate(mask, SHORE_TRANSITION, EIGHT_NEIGHBOURS, 1, COASTAL, SHORE_TRANSITION)
    dilate(mask, INLANDWATER, EIGHT_NEIGHBOURS, 1, COASTAL, SHORE_INLANDWATER)

# count distance from ocean in shore of transition zone
for i in range(transition_depth):
    dilate(mask, OCEAN if i == 0 else COASTAL+i, EIGHT_NEIGHBOURS, 1, SHORE_TRANSITION, COASTAL+i+1)

# revert inland water shore and unreached transition zone shore
mask[(mask==SHORE_INLANDWATER)|(mask==SHORE_TRANSITION)] = COASTAL

da = xr.DataArray(mask.reshape((1, *mask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da[:,1455:1555,1055:1155].plot()

In [ ]:
print("write s2w mask ...")

# write result to s2w mask
s2w = xr.DataArray(mask.reshape((1, hr_height, hr_width)), coords=hr.coords, dims=hr.dims, attrs=hr.attrs)
s2w.rio.to_raster(s2w_mask_path, compress='LZW', tiled=True)

print("counting pixel classes ...")

land_count = np.count_nonzero(mask>=128)
inlandwater_count = np.count_nonzero((mask<128) & (mask > 64))
ocean_count = np.count_nonzero(mask<=64)
with open(countslist_path, "a") as f:
    f.write(f"{s2w_mask_path}\t{ocean_count}\t{inlandwater_count}\t{land_count}\n")

print(f"\n{land_count=}")
print(f"{inlandwater_count=}")
print(f"{ocean_count=}\n")

print(s2w_mask_path)

da = xr.DataArray(mask.reshape((1, *mask.shape)), dims=hr.dims, coords=hr.coords, attrs=hr.attrs)
da.plot()

### done